In [1]:
import pandas as pd
import requests

In [2]:
APIKEY = None
with open('apikey.txt', 'r') as _apikey:
    APIKEY = _apikey.read()
    _apikey.close()

https://www.alphavantage.co/documentation/

Example: 

https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220410T0130&limit=100&apikey=

https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220510T0130&topics=finacail_market,earnings&limit=10&apikey=

In [3]:
API_URL_PRICE_DAILY = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&outputsize=full&apikey={APIKEY}"

In [4]:
with requests.Session() as s:
    download = s.get(API_URL_PRICE_DAILY)
    df_price = pd.DataFrame(download.json()['Time Series FX (Daily)']).T
    df_price.columns = [i.split()[1] for i in df_price.columns.values]
    df_price.sort_index(inplace=True)
df_price.to_csv(f"data/APLHAVANTAGE_EURUSD.csv")
df_price

,open,high,low,close
2004-02-09,1.26760,1.27610,1.26150,1.26960
2004-02-10,1.26830,1.27880,1.26630,1.26800
2004-02-11,1.27030,1.28340,1.26460,1.28250
2004-02-12,1.28100,1.28460,1.27830,1.28030
2004-02-13,1.27900,1.28910,1.27110,1.27400
...,...,...,...,...
2023-04-03,1.08418,1.09166,1.07870,1.09006
2023-04-04,1.09011,1.09733,1.08810,1.09540
2023-04-05,1.09549,1.09696,1.08900,1.09057
2023-04-06,1.09058,1.09376,1.08830,1.09176


In [5]:
dates = df_price.index.values
dates_for_news = [i.replace('-','')+'T2350' for i in dates]

In [6]:
link = "https://www.alphavantage.co/query?"
func = "function=NEWS_SENTIMENT"
tickers="tickers=FOREX:EUR"
time_from = "time_from=20220310T0130" 
time_to = "time_to=20220410T0130"
url_apikey = "apikey={APIKEY}"

API_URL_NEWS = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220410T0130&apikey={APIKEY}"


In [32]:
def parse_news_data(_urlNews: str):
    with requests.Session() as s:
        download = s.get(_urlNews)
        df_news = pd.json_normalize(download.json()['feed'])
        # df_news['authors_clear'] = df_news.apply(lambda x: x.authors, axis=1)
    return df_news

In [33]:
df = parse_news_data(API_URL_NEWS)

In [34]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,"Friday, April 8. Russia's War On Ukraine: News...",https://www.forbes.com/sites/katyasoldak/2022/...,20220409T005955,[Katya Soldak],"Dispatches from Ukraine, provided by Forbes Uk...",https://imageio.forbes.com/specials-images/ima...,Forbes,Business,www.forbes.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.028179,Neutral,"[{'ticker': 'VOD', 'relevance_score': '0.05925..."
1,Has the pound become emerging-market money?,https://www.economist.com/finance-and-economic...,20220409T000000,[],Why the British currency has become more vulne...,https://www.economist.com/img/b/608/814/90/sit...,The Economist,Finance,www.economist.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.140854,Neutral,"[{'ticker': 'BAC', 'relevance_score': '0.06279..."
2,ECB executive board member talks about current...,https://cointelegraph.com/news/ecb-executive-b...,20220408T211500,[],Fabio Panetta outlined recent findings and rem...,https://images.cointelegraph.com/images/1200_a...,Cointelegraph,n/a,cointelegraph.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.002530,Neutral,"[{'ticker': 'CBSU', 'relevance_score': '0.5691..."
3,Speculators pare back net long U.S. dollar bet...,https://www.reuters.com/business/speculators-p...,20220408T203800,[Reuters],Speculators pare back net long US dollar bets ...,https://cloudfront-us-east-2.images.arcpublish...,Reuters,Business,www.reuters.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.009313,Neutral,"[{'ticker': 'FOREX:CHF', 'relevance_score': '0..."
4,"Closing prices for crude oil, gold and other c...",https://apnews.com/article/business-financial-...,20220408T195358,[],"Closing prices for crude oil, gold and other c...",https://storage.googleapis.com/afs-prod/media/...,Associated Press,Markets,apnews.com,[],0.000211,Neutral,"[{'ticker': 'FOREX:JPY', 'relevance_score': '0..."
5,"Forex reserves slide $11.17 bn to $606.475 bn,...",https://www.business-standard.com/article/fina...,20220408T182200,[Press Trust of India],For the previous reporting week ended March 25...,https://bsmedia.business-standard.com/_media/b...,Business Standard,Finance,www.business-standard.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.003070,Neutral,"[{'ticker': 'FOREX:JPY', 'relevance_score': '0..."
6,Switzerland-based SIG considering investment i...,https://www.business-standard.com/article/comp...,20220408T163300,[Press Trust of India],After setting its local office in Gurugram in ...,https://bsmedia.business-standard.com/_media/b...,Business Standard,Companies,www.business-standard.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.135070,Neutral,"[{'ticker': 'KO', 'relevance_score': '0.101539..."
7,Caisse Française de Financement Local: EMTN 20...,https://www.globenewswire.com/news-release/202...,20220408T163200,[Caisse Française de Financement Local],"Issy les Moulineaux, 8 April 2022",https://ml-eu.globenewswire.com/Resource/Downl...,GlobeNewswire,n/a,www.globenewswire.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.000406,Neutral,"[{'ticker': 'FOREX:EUR', 'relevance_score': '0..."
8,Benchmark Treasury yield hits 3-year high; dol...,https://www.financialexpress.com/market/benchm...,20220408T155818,[],This week's release of the minutes from the Fe...,https://www.financialexpress.com/wp-content/up...,The Financial Express,n/a,www.financialexpress.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.088774,Neutral,"[{'ticker': 'WU', 'relevance_score': '0.104002..."
9,Draft resolutions prepared by the Board for th...,https://www.globenewswire.com/news-release/202...,20220408T155100,[Invalda INVL],The draft resolutions prepared by the Board of...,https://ml-eu.globenewswire.com/Resource/Downl...,GlobeNewswire,n/a,www.globenewswire.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.001418,Ne